In [ ]:
                            ############### IMPORTATION DES PACKAGES #################
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.calibration import calibration_curve
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

from lime.lime_tabular import LimeTabularExplainer
from catboost import CatBoostClassifier, Pool

In [ ]:
                    ################# IMPORTATION DES DATASETS ET CONCATENATION #####################

# Chargement des fichiers PTB
ptb_abnormal = pd.read_csv('ptbdb_abnormal.csv', header=None)
ptb_normal = pd.read_csv('ptbdb_normal.csv', header=None)
ptb = pd.concat([ptb_normal, ptb_abnormal], ignore_index=True, sort=False)
ptb.rename(columns={187 :'class'}, inplace=True)
    
# Chargement des fichiers MITBIH
mit_train = pd.read_csv('mitbih_train.csv', header=None)
mit_test = pd.read_csv('mitbih_test.csv', header=None)
mit = pd.concat([mit_train, mit_test], ignore_index=True, sort=False)
mit.rename(columns={187 :'class'}, inplace=True)
mit['class'].replace({2:1, 3:1, 4:1}, inplace=True)
    
# Concaténation des deux ensembles de données
data = pd.concat([ptb, mit], ignore_index=True, sort=False)
    
# Variables cibles
y = data['class'].copy()
    
# Features
X = data.drop('class', axis=1).copy()

In [ ]:
                                    ################# MODÈLE CATBOOST 1 ################
                                    
# Séparation en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1, shuffle=True)

# Define the model with custom hyperparameters
model = CatBoostClassifier(iterations=4000, learning_rate=0.08, depth=8, metric_period=100)

# Fit the model to the training data
model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=100)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Classification report 
print(classification_report(y_test, y_pred))

In [ ]:
                              ####################### MODÈLE CATBOOST 2 #########################

# Séparation en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1, shuffle=True)

# Spécifier les hyperparamètres du modèle CatBoostClassifier
params = {
    'iterations': 2000,
    'learning_rate': 0.08,
    'depth': 8,
    'l2_leaf_reg': 3,
    'border_count': 64, 
    'random_seed': 42,
    'eval_metric': 'Accuracy', 
    'metric_period': 100
}

# Créer un modèle CatBoostClassifier avec les hyperparamètres spécifiés
model = CatBoostClassifier(**params)

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=100)

# Faire des prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Obtenir les probabilités de prédiction pour la classe positive
y_proba = model.predict_proba(X_test)[:, 1]

# Classification report 
print(classification_report(y_test, y_pred))

In [ ]:
                            ############## INTERPRETABILITÉ AVEC PACKAGE LIME #################

# Utiliser la bibliothèque LIME pour expliquer les prédictions du modèle pour une observation spécifique
explainer = LimeTabularExplainer(X_train.values, feature_names=X_train.columns, 
                                 class_names=['0 : normal', '1 : abnormal'], 
                                 discretize_continuous=True)

exp = explainer.explain_instance(X_test.iloc[0,:].values, model.predict_proba, num_features=5)
exp.show_in_notebook()

In [ ]:
                        ############ COURBE D'APPRENTISSAGE POUR UN POOL DE TRAIN ET TEST ####################                      

# Définir les données d'entraînement et de test
train_data = Pool(X_train, y_train)
test_data = Pool(X_test, y_test)

# Tracer les courbes d'apprentissage
plt.plot(model.get_evals_result()['learn']['Accuracy'], label='Train set')
plt.plot(model.get_evals_result()['validation']['Accuracy'], label='Test set')
plt.title('Learning Curve CatBoost')
plt.xlabel('Training Set Size')
plt.ylabel('Accuracy score')
plt.legend()
plt.show(); 

In [ ]:
                                    ################# COURBE DE CALIBRATION ###################

# Calculer la courbe de calibration
fraction_of_positives, mean_predicted_value = calibration_curve(y_test, y_proba, n_bins=10)

# Tracer la courbe de calibration
plt.plot(mean_predicted_value, fraction_of_positives, "s-", label="Model", color='red')

# Tracer la diagonale parfaite
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")

plt.xlabel("Mean predicted value")
plt.ylabel("Fraction of positives")
plt.ylim([-0.05, 1.05])
plt.legend(loc="lower right")
plt.title('Calibration Curve')
plt.show();

In [ ]:
                                ############ INTERPRÉTABILITÉ : FEATURE IMPORTANCE ##################

feat_importances = pd.Series(model.feature_importances_, index=X_train.columns)
feat_importances.nlargest(10).plot(kind='barh', color='red')
plt.title('Feature importance')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show(); 

In [ ]:
                                            ############# MODÈLE CATBOOST 3 ##############

model = CatBoostClassifier(iterations=1000, 
                           learning_rate=0.1520172767474338, 
                           depth=9, 
                           l2_leaf_reg=1.4205108576277599, 
                           bagging_temperature=1.123731327207984, 
                           random_strength=0.19335447144110063,
                           eval_metric='Logloss',
                           metric_period=100,
                           verbose=100)

model.fit(X_train, y_train)

y_proba = model.predict_proba(X_test)[:,1]
auc = roc_auc_score(y_test, y_proba)
print('AUC on test set:', auc)

# Prédire les labels pour l'ensemble de test
y_pred = model.predict(X_test)

# Afficher le classification report
print(classification_report(y_test, y_pred))

In [ ]:
                                ############# MATRICE DE CONFUSION ################

# Calculer la matrice de confusion
cm = confusion_matrix(y_test, y_pred)

# Créer une heatmap pour visualiser la matrice de confusion
plt.figure()
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Classe prédite")
plt.ylabel("Classe réelle")
plt.title("Matrice de confusion")
plt.show();

### Hyperparamètres qui peuvent être utiles dans le cadre de la classification avec CatBoost :

> learning_rate : la vitesse à laquelle le modèle ajuste les poids des features lors de l'apprentissage. Une valeur plus élevée peut entraîner une convergence plus rapide mais peut également entraîner un surajustement.

> max_depth : la profondeur maximale de l'arbre de décision. Une valeur plus élevée peut entraîner un surajustement.

> l2_leaf_reg : le coefficient de régularisation L2 pour les feuilles de l'arbre de décision. Une valeur plus élevée peut aider à prévenir le surajustement.

> bagging_temperature : contrôle l'agressivité du bagging. Des valeurs plus élevées peuvent aider à prévenir le surajustement.

> border_count : le nombre de subdivisions à utiliser pour les features catégorielles. Des valeurs plus élevées peuvent améliorer la performance, mais cela peut également augmenter le temps d'entraînement.

> learning_curve : le taux d'apprentissage est un hyperparamètre important qui contrôle la vitesse à laquelle le modèle apprend à partir des données. Un taux d'apprentissage trop élevé peut entraîner une instabilité du modèle, tandis qu'un taux d'apprentissage trop faible peut entraîner une convergence lente. C'est donc un hyperparamètre important à optimiser.

In [ ]:
                                ############## MODÈLE CATBOOST 4 ##############

# Diviser les données en ensemble de train et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3.5 ; 4 ; 
# Spécifier les poids des classes
class_weights = {0: 1, 1: 3}

# Spécifier les hyperparamètres du modèle CatBoostClassifier
params = {
    'iterations': 2000,
    'learning_rate': 0.08,
    'depth': 8,
    'l2_leaf_reg': 8,
    'bagging_temperature': 4, 
    'border_count': 64,  
    'random_seed': 42,
    'eval_metric': 'Accuracy', 
    'metric_period': 100,
    'class_weights': class_weights
}

# Créer un modèle CatBoostClassifier avec les hyperparamètres spécifiés
model = CatBoostClassifier(**params)

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=100)

# Faire des prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Classification report 
print(classification_report(y_test, y_pred))

In [ ]:
                            ################ COURBE D'APPRENTISSAGE POUR UN POOL DE TRAIN ET TEST ################ 

# Définir les données d'entraînement et de test
train_data = Pool(X_train, y_train)
test_data = Pool(X_test, y_test)

# Tracer les courbes d'apprentissage
plt.plot(model.get_evals_result()['learn']['Accuracy'], label='Train set')
plt.plot(model.get_evals_result()['validation']['Accuracy'], label='Test set')
plt.title('Learning Curve CatBoost')
plt.xlabel('Training Set Size')
plt.ylabel('Accuracy score')
plt.legend()
plt.show();

Ce code utilise Optuna pour effectuer une optimisation hyperparamétrique d'un modèle CatBoostClassifier. Il utilise la validation croisée pour estimer la précision du modèle, et recherche les valeurs optimales pour les hyperparamètres suivants :

- objective: la fonction d'objectif utilisée pour l'entraînement du modèle (log-loss ou cross-entropy)
- colsample_bylevel: la fraction de colonnes à utiliser à chaque niveau de l'arbre
- depth: la profondeur maximale de l'arbre
- boosting_type: le type de boosting (ordered ou plain)
- bootstrap_type: le type de bootstrap (Bayesian, Bernoulli, ou MVS)
- bagging_temperature: la température pour le bootstrap bayésien
- subsample: la fraction d'observations à utiliser pour le bootstrap bernoulli


In [ ]:
                                    ############# MODÈLE CATBOOST 5 #################

# Diviser les données en ensemble de train et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
 
# Spécifier les poids des classes
class_weights = {0: 1, 1: 3}

# Spécifier les hyperparamètres du modèle CatBoostClassifier
params = {
    'iterations': 3000,
    'objective': 'Logloss',
    'learning_rate': 0.08,
    'colsample_bylevel': 0.022546586750624834,
    'depth': 12,
    'l2_leaf_reg': 8,
    'boosting_type': 'Plain',
    'bootstrap_type': 'Bayesian',
    'bagging_temperature': 9.954611183129638,  
    'eval_metric': 'Accuracy', 
    'metric_period': 100,
    'class_weights': class_weights 
}

# Créer un modèle CatBoostClassifier avec les hyperparamètres spécifiés
model = CatBoostClassifier(**params)

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=100)

# Faire des prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Obtenir les probabilités de prédiction pour la classe positive
y_proba = model.predict_proba(X_test)[:, 1]

# Classification report 
print(classification_report(y_test, y_pred))

In [ ]:
                                            ############ COURBE DE CALIBRATION ###############
                                            
# Calculer la courbe de calibration
fraction_of_positives, mean_predicted_value = calibration_curve(y_test, y_proba, n_bins=10)

# Tracer la courbe de calibration
plt.plot(mean_predicted_value, fraction_of_positives, "s-", label="Model", color='red')

# Tracer la diagonale parfaite
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")

plt.xlabel("Mean predicted value")
plt.ylabel("Fraction of positives")
plt.ylim([-0.05, 1.05])
plt.legend(loc="lower right")
plt.title('Calibration Curve')
plt.show();

In [ ]:
                        ################ COURBE D'APPRENTISSAGE POUR UN POOL DE TRAIN ET TEST ################ 

# Définir les données d'entraînement et de test
train_data = Pool(X_train, y_train)
test_data = Pool(X_test, y_test)

# Tracer les courbes d'apprentissage
plt.plot(model.get_evals_result()['learn']['Accuracy'], label='Train set')
plt.plot(model.get_evals_result()['validation']['Accuracy'], label='Test set')
plt.title('Learning Curve CatBoost')
plt.xlabel('Training Set Size')
plt.ylabel('Accuracy score')
plt.legend()
plt.show();

In [ ]:
                                    ################ MATRICE DE CONFUSION #################

# Calculer la matrice de confusion
cm = confusion_matrix(y_test, y_pred)

# Créer une heatmap pour visualiser la matrice de confusion
plt.figure()
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Classe prédite")
plt.ylabel("Classe réelle")
plt.title("Matrice de confusion")
plt.show();